In [32]:
import time
import pandas as pd
import numpy as np
import requests
import json
from selenium import webdriver
import bs4
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [152]:
client_id = 'AJb44uAUgBS5u1gIdz0P'
client_secret = 'ThMXBzW9HG'

url = 'https://openapi.naver.com/v1/search/news?'

header = {"X-Naver-Client-Id":client_id,"X-Naver-Client-secret":client_secret}
keyword = '삼성전자'# 검색어
display = 100 # 1~100
start = 1 # 1~1000
sort = 'sim' # sim, date
params = {'query':keyword , 'display':display, 'start':start, 'sort':sort}
req = requests.get(url, params = params, headers = header)
js = req.json()

In [153]:
js['items'][0]

{'title': '<b>삼성전자</b>, 에너지 취약계층 대상 에어컨 1만대 공급',
 'originallink': 'http://www.newsis.com/view/?id=NISX20220426_0001847657&cID=13001&pID=13000',
 'link': 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=003&aid=0011145974',
 'description': "<b>삼성전자</b>가 에너지 취약 가구의 건강한 여름 나기를 돕기 위해 <b>삼성</b> 에어컨 '벽걸이 와이드'를 공급한다. <b>삼성전자</b>는 한국에너지재단이 주관하는 '2022년 저소득층 에너지효율개선사업'에 냉방기기 보급 사업자로... ",
 'pubDate': 'Tue, 26 Apr 2022 11:00:00 +0900'}

In [172]:
client_id = 'AJb44uAUgBS5u1gIdz0P'
client_secret = 'ThMXBzW9HG'

url = 'https://openapi.naver.com/v1/search/news?'

header = {"X-Naver-Client-Id":client_id,"X-Naver-Client-secret":client_secret}

symbol = '카카오' # 종목
word_list = ['주식','뉴스','개인','기관','2022년','2021년','2020년','실적','분석'] # 검색어
display = 100 # 1~100
start = 1 # 1~1000
sort = 'sim' # sim(정확도), date(시간순)

title_list = []
link_list = []
date_list = []
for word in word_list:
    keyword = symbol + ' ' + word
    start = 1
    for _ in range(10):
        params = {'query':keyword , 'display':display, 'start':start, 'sort':sort}
        req = requests.get(url, params = params, headers = header)
        js = req.json()
        title_list += [js['items'][i]['title'] for i in range(100) if 'news.naver' in js['items'][i]['link']]
        link_list += [js['items'][i]['link'] for i in range(100) if 'news.naver' in js['items'][i]['link']]
        date_list += [js['items'][i]['pubDate'] for i in range(100) if 'news.naver' in js['items'][i]['link']]
        start += 100
    print(f'{keyword} 검색 뉴스기사 크롤링 완료')

카카오 주식 검색 뉴스기사 크롤링 완료
카카오 뉴스 검색 뉴스기사 크롤링 완료
카카오 개인 검색 뉴스기사 크롤링 완료
카카오 기관 검색 뉴스기사 크롤링 완료
카카오 2022년 검색 뉴스기사 크롤링 완료
카카오 2021년 검색 뉴스기사 크롤링 완료
카카오 2020년 검색 뉴스기사 크롤링 완료
카카오 실적 검색 뉴스기사 크롤링 완료
카카오 분석 검색 뉴스기사 크롤링 완료


In [174]:
print(len(word_list)*1000)
print('네이버기사',len(title_list))
print('안겹치는 링크',len(set(link_list)))

9000
네이버기사 4342
안겹치는 링크 3340


In [148]:
# for i in Counter(date_list):
#     if Counter(date_list)[i] > 1:
#         print(i,':',Counter(date_list)[i])
# for i in Counter(title_list):
#     if Counter(title_list)[i] > 1:
#         print(i,':',Counter(title_list)[i])

In [175]:
df = pd.DataFrame()
df['title'] = title_list
df['link'] = link_list
df['date'] = date_list

In [176]:
df

,title,link,date
0,<b>주식</b>선물에 해외 소수점거래… <b>카카오</b>페이증권 vs 토스증권 ...,https://news.naver.com/main/read.naver?mode=LS...,"Mon, 25 Apr 2022 06:20:00 +0900"
1,"<b>카카오</b>페이證, 美 우량 기업 <b>주식</b> 제공 이벤트 실시",https://news.naver.com/main/read.naver?mode=LS...,"Mon, 25 Apr 2022 12:18:00 +0900"
2,"<b>카카오</b>페이증권, MTS 정식 출시 기념 무료 <b>주식</b> 이벤트",https://news.naver.com/main/read.naver?mode=LS...,"Mon, 25 Apr 2022 10:42:00 +0900"
3,"<b>카카오</b>페이증권, MTS 정식 출시 기념 <b>주식</b> 증정 이벤트",https://news.naver.com/main/read.naver?mode=LS...,"Mon, 25 Apr 2022 10:56:00 +0900"
4,"<b>카카오</b>페이증권, ‘100% 당첨’ 해외<b>주식</b> 제공 이벤트 진행",https://news.naver.com/main/read.naver?mode=LS...,"Mon, 25 Apr 2022 10:58:00 +0900"
...,...,...,...
4337,공정위 새해도 '플랫폼 갑질' 겨눈다…구글·<b>카카오</b>·쿠팡 사정권,https://news.naver.com/main/read.naver?mode=LS...,"Mon, 03 Jan 2022 05:30:00 +0900"
4338,"공정위, 올해도 구글·<b>카카오</b>·쿠팡의 ‘갑질’ 겨눈다… 법·제도 개선 착수",https://news.naver.com/main/read.naver?mode=LS...,"Mon, 03 Jan 2022 06:42:00 +0900"
4339,공정위 새해도 '플랫폼 갑질' 겨눈다…구글·<b>카카오</b>·쿠팡 사정권,https://news.naver.com/main/read.naver?mode=LS...,"Mon, 03 Jan 2022 12:03:00 +0900"
4340,공정위 새해도 `플랫폼 갑질` 칼날…구글·<b>카카오</b>·쿠팡 겨냥,https://news.naver.com/main/read.naver?mode=LS...,"Mon, 03 Jan 2022 14:56:00 +0900"


In [177]:
df = df.groupby('link')

In [179]:
date_list

['Mon, 25 Apr 2022 06:20:00 +0900',
 'Mon, 25 Apr 2022 12:18:00 +0900',
 'Mon, 25 Apr 2022 10:42:00 +0900',
 'Mon, 25 Apr 2022 10:56:00 +0900',
 'Mon, 25 Apr 2022 10:58:00 +0900',
 'Sun, 24 Apr 2022 07:02:00 +0900',
 'Mon, 25 Apr 2022 11:27:00 +0900',
 'Mon, 25 Apr 2022 11:23:00 +0900',
 'Mon, 25 Apr 2022 14:49:00 +0900',
 'Mon, 25 Apr 2022 14:05:00 +0900',
 'Mon, 25 Apr 2022 10:46:00 +0900',
 'Mon, 25 Apr 2022 11:35:00 +0900',
 'Sun, 24 Apr 2022 08:58:00 +0900',
 'Mon, 25 Apr 2022 09:33:00 +0900',
 'Mon, 25 Apr 2022 10:42:00 +0900',
 'Sun, 24 Apr 2022 09:22:00 +0900',
 'Mon, 25 Apr 2022 15:00:00 +0900',
 'Mon, 25 Apr 2022 13:09:00 +0900',
 'Tue, 26 Apr 2022 11:42:00 +0900',
 'Mon, 25 Apr 2022 10:17:00 +0900',
 'Mon, 25 Apr 2022 14:30:00 +0900',
 'Mon, 25 Apr 2022 15:46:00 +0900',
 'Mon, 25 Apr 2022 14:42:00 +0900',
 'Sun, 24 Apr 2022 15:23:00 +0900',
 'Sun, 24 Apr 2022 15:43:00 +0900',
 'Tue, 26 Apr 2022 08:32:00 +0900',
 'Mon, 25 Apr 2022 10:23:00 +0900',
 'Mon, 25 Apr 2022 18:35:00 